In [52]:
#langraph agents
import datasets
from langchain.docstore.document import Document

guest_dataset = datasets.load_dataset("agents-course/unit3-invitees",split='train')

docs = [Document(
    page_content="\n".join([
        f"Name : {guest['name']}",
        f"Relation: {guest['relation']}",
        f"Description: {guest['description']}",
        f"Email: {guest['email']}"
    ]),
    metadata={'name':guest['name']}
)
    for guest in guest_dataset]

In [53]:
docs

[Document(metadata={'name': 'Ada Lovelace'}, page_content="Name : Ada Lovelace\nRelation: best friend\nDescription: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine.\nEmail: ada.lovelace@example.com"),
 Document(metadata={'name': 'Dr. Nikola Tesla'}, page_content="Name : Dr. Nikola Tesla\nRelation: old friend from university days\nDescription: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about pigeons, so that might make for good small talk.\nEmail: nikola.tesla@gmail.com"),
 Document(metadata={'name': 'Marie Curie'}, page_content='Name : Marie Curie\nRelation: no relation\nDescription: Marie Curie was a groundbreaking physici

* Load the dataset and store the dataset in list of docyments.

#

In [64]:
#retiever tool
from langchain_community.retrievers import BM25Retriever
from langchain.tools import Tool

bm25_retriever = BM25Retriever.from_documents(docs)

def extract_text(query: str) ->str:
    """Retrieves detailed information about gala guests based on their name and relation."""
    results = bm25_retriever.invoke(query)
    if results:
        return "\n\n".join([doc.page_content for doc in results[:3]])
    else:
        return "No matching guest information found."
    
guest_info_tool = Tool(
    name="guest_info_retriever",
    func=extract_text,
    description="Retrieves detailed information about gala guests based on their name or relation."
)

In [79]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage,SystemMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun


In [ ]:
# Define Nodes
import os 
os.environ['OPENAI_API_KEY']='**'


In [127]:
search = DuckDuckGoSearchRun()

In [128]:
llm = ChatOpenAI(model='gpt-4o')

tools = [guest_info_tool,search]
chat_with_tools = llm.bind_tools(tools)


In [129]:
#state
class AgentState(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

def assistant(state:AgentState):
    
    return {
        "messages":[chat_with_tools.invoke(state['messages'])]
    }

In [130]:
#graph
builder = StateGraph(AgentState)
builder.add_node("assistant",assistant)
builder.add_node("tools",ToolNode(tools))

builder.add_edge(START,"assistant")

builder.add_conditional_edges("assistant",tools_condition)

builder.add_edge("tools","assistant")
alfred = builder.compile()


In [131]:
messages = [HumanMessage(content="Tell me about our guest named 'Lady Ada Lovelace'.")]
response = alfred.invoke({"messages": messages})

print("🎩 Alfred's Response:")


🎩 Alfred's Response:


In [133]:
for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

Tell me about our guest named 'Lady Ada Lovelace'.
================================== Ai Message ==================================
Tool Calls:
  guest_info_retriever (call_vkQYHCNOqTiFmxRl0QdTzYRa)
 Call ID: call_vkQYHCNOqTiFmxRl0QdTzYRa
  Args:
    __arg1: Lady Ada Lovelace
================================= Tool Message =================================
Name: guest_info_retriever

Name : Ada Lovelace
Relation: best friend
Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine.
Email: ada.lovelace@example.com

Name : Marie Curie
Relation: no relation
Description: Marie Curie was a groundbreaking physicist and chemist, famous for her research on radioactivity.
Email: marie.curie@example.com

Na

In [134]:
response = alfred.invoke({"messages":"I need to speak with 'Dr. Nikola Tesla' about recent advancements in **wireless energy**. Can you help me prepare for this conversation ?"})

print("🎩 Alfred's Response:")


🎩 Alfred's Response:


In [135]:
for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

I need to speak with 'Dr. Nikola Tesla' about recent advancements in **wireless energy**. Can you help me prepare for this conversation ?
================================== Ai Message ==================================
Tool Calls:
  guest_info_retriever (call_Jjdnmq1XNZEHu6tm0KLt7UZU)
 Call ID: call_Jjdnmq1XNZEHu6tm0KLt7UZU
  Args:
    __arg1: Dr. Nikola Tesla
  duckduckgo_search (call_Ahx1IJrT0HlAukaqsAHQKwLk)
 Call ID: call_Ahx1IJrT0HlAukaqsAHQKwLk
  Args:
    query: recent advancements in wireless energy
================================= Tool Message =================================
Name: guest_info_retriever

Name : Dr. Nikola Tesla
Relation: old friend from university days
Description: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about pigeons,